# RL Experiments for Dyadic Strategies

#### Exp 1: change c_err/c_push, c_push/c_pull. individual difference
#### Exp 2: change discretization bins

In [1]:
# Write the environment in the same style as OpenAI Gym

# Sample environment
# https://github.com/upb-lea/gym-electric-motor/blob/master/gym_electric_motor/core.py

#PEP8 Style Guide
# https://www.python.org/dev/peps/pep-0008/

In [2]:
# Imports for each experiment
import sys, os
import importlib, time

lib_dir = 'lib/'

sys.path.append(lib_dir)

In [5]:
import configparser
from helper_funcs import rk4_step
import trajectory_tools
# importlib.reload(trajectory_tools)
from trajectory_tools import Trajectory
import numpy as np

class PhysicalDyads():
    
    allowable_keys = ('obj_mass', 'obj_fric', 'tstep', 'duration', 'failure_error',
                      'f_bound', 'max_ref', 'max_freq')

    # internal methods
    def __init__(self, env_id=None, seed_=None, config_file=None, **kwargs):
        # Returns an instantiated dyadic game environment. 
        # Given that the environment is dynamic, the current time step can be queried
        # from variable "step_i".
        
        # env_id can be either 'soft' or 'hard'.
        # 'soft' corresponds to soft force constraints, i.e. the task doesn't end 
        # (the carried object does not break) if normal force range is violated.
        # 'hard' is the other case.
        
        
        # There are 2 mechanisms to set/modify env parameters: config file, kwargs.
        # config file is the address to the INI config file.
            # - The file does not need to contain all parameters but only those that are modified
            # - Sections are not important
            # - Only scalars and strings are allowed as values.
        
        # Read params from config file.
        Config = configparser.ConfigParser()
        default_config = 'configs/env_config.ini'
        Config.read(default_config) # read the default config file.
        
        for section in Config.sections():
            for key in Config.options(section):
                if key in self.allowable_keys:
                    val = Config.get(section, key)
                    try:
                        val = float(val)
                    except:
                        pass
                    setattr(self, key, val)       
        
        if config_file!=None:
            Config = configparser.ConfigParser()
            Config.read(config_file) # read the default config file.
        
            for section in Config.sections():
                for key in Config.options(section):
                    if key in self.allowable_keys:
                        val = Config.get(section, key)
                        try:
                            val = float(val)
                        except:
                            pass
                        setattr(self, key, val)
        
        # Read the kwargs and override params from config file.
        for key in kwargs:
            if key in self.allowable_keys:
                setattr(self, key, kwargs[key])

    
    
    
        self.max_err = self.max_ref*self.failure_error
        # Initialize state variables
        self.step_i =0
        self.x, self.v, self.f1_old, self.f2_old = 0., 0., 0., 0.
        self.done = False
        self.traj_creator = Trajectory(self.tstep, seed_=seed_)
        self.traj_time, self.traj = None, None
        self._max_episode_steps = int(self.duration/self.tstep)
        
    
    def get_time(self):
        return self.step_i
    
    def get_episode_duration(self):
        return self.duration
    
    def _dynamic_sys(self, x, u, t=0):
        return np.array([x[1],(-self.obj_fric*x[1]+x[2])/self.obj_mass, u])
    
    def _update_state(self, net_f, net_df, t, tstep):

        obj_state = [self.x, self.v, net_f]
        self.x, self.v, self.net_f = rk4_step(self._dynamic_sys, obj_state, net_df, t, tstep)
    

    def _update_fn(self, f1, f2):
        
        # Returns normal force and its first derivative
        
        raise NotImplementedError
    
    
    @staticmethod
    def reward(r, x):
        err = -abs(r-x)
        return err
        
    
    # interface functions
    
#     def seed(self, seed_):
#         # Make sure the all the random generators in the environment use this seed.
#         np.random.seed(seed_)
#         self.traj_creator.s
#         # Do not use Python's native random generator for consistency
#         raise NotImplementedError
        
    def reset(self, renew_traj=False):
        # Resets the initial state and the reference trajectories. 
        # Call this function before a new episode starts.
#         return initial observations
        if (renew_traj is False and self.traj is None) or \
        (renew_traj is True):
            # generate a traj
            self.traj_time, self.traj = self.traj_creator.generate_random(self.duration, \
                n_traj=1, max_amp=self.max_ref, traj_max_f=self.max_freq, rel_amps=None, fixed_effort=True, \
                obj_mass=self.obj_mass, obj_fric=self.obj_fric, n_deriv=1, ret_specs=False)
        
        self.x, self.v  = self.traj[0][0], self.traj[1][0]
        self.step_i = 0
        self.f1_old, self.f2_old = 0., 0.
        self.done = False
        
        return [self.x, self.v, self.x, self.v, 0., 0.]
            
    
    def is_terminal(self, r):
        if self.step_i== self._max_episode_steps-1:
            return True 
        # If the positional error is too large, end the episode
        if r-self.x > self.max_err:
            return True
        return False
        
        
    def step(self, action):
#         return (reference, state, normal force), reward, done, _
    # action is a tuple of two forces
    # Calling step() after the episode is done will return None.
    
        if self.done is True:
            print('Warning: Episode has ended. Reset the environment!')
            return None
        
        self.step_i +=1
        t = self.traj_time[self.step_i]
        r, r_dot = self.traj[0][self.step_i], self.traj[1][self.step_i] #Set reference
        
        self.done = self.is_terminal(r) # Check terminal
        
        # Update object state
        f1, f2 = action
        net_f = f1-f2
        net_df = net_f - (self.f1_old-self.f2_old)
        self._update_state(net_f, net_df, t, self.tstep)
        
        # Calculate normal force and its derivative
        fn = min(f1, f2); fn_old = min(self.f1_old, self.f2_old);
        fn_dot = (fn-fn_old)/self.tstep 
        
        self.f1_old = f1; self.f2_old = f2;
        
#         self.done = self.is_terminal() # Check terminal due to new positional error 
        
        return [r, r_dot, self.x, self.v, fn, fn_dot], PhysicalDyads.reward(r,self.x), self.done, None
    
    
    def render(self):
        # Not schedulled to be implemented at this phase. 
        raise NotImplementedError
        
    def close(self):
        pass

In [6]:
env = PhysicalDyads(seed_=1234)

In [7]:
env.reset(renew_traj=True)
# traj1 = env.traj
# env.reset(renew_traj=True)
# traj2 = env.traj

[-0.20353710364613078,
 -0.6151401660307525,
 -0.20353710364613078,
 -0.6151401660307525,
 0.0,
 0.0]

In [8]:
t0 = time.time()
done=False
while done is not True:
    observations, reward, done, _ = env.step((1.1, 0.3))
#     print(reward)
#     print(observations[4], observations[5])
elapsed_t = time.time() - t0
print('elapsed_t: ', elapsed_t)

elapsed_t:  0.03722119331359863


In [11]:
# env interface
# the first observation comes from the env.reset
# the next ones come from env.step

In [17]:
class TDAgent:
    # Developed for being trained on the PhysicalDyads environment, using n-step Sarsa algorithm.
    
    # Steps from observation to action:
    # 1- Calculate features: e, e', f, f' from observations={r, r', x, x', fn, f'n}
    # 2- Discretize features and Find the corresponding element in Q table
    # 3- Apply Greedy to the corresponding row in Q table to get role.
    # 4- Convert role to force. (PID, force bounds)
    # 5- Store the outcome of the action for future use
    
    # agents have a planner, which outputs role.
# roles are translated to df, using the PIDs from before.
# bound df. calc f and bound it.
# (f1, f2) is fed to env.step()
# env.step() returns observation={r, r', x, x', fn, f'n}, reward={-abs(r-x)}, done, info

        # bins size => ss dimensions => q table, 
        # not sure if we should keep policy table
        # epsilon, gamma
        # or we could assume env is a global variable.
    allowable_keys = ('c_err', 'c_push', 'c_pull', 'n_bins',  'tstep', 'duration','dotf_bound', 'f_bound', 'max_ref')
    
    # internal methods
    def __init__(self, agent_id=None, seed_=None, config_file=None, **kwargs):
        
        # config file is the address to the INI config file.
            # - The file does not need to contain all parameters but only those that are modified
            # - Sections are not important
            # - Only scalars and strings are allowed as values.
        
        # Read params from config file.
        Config = configparser.ConfigParser()
        default_config = 'configs/agent_config.ini'
        Config.read(default_config) # read the default config file.
        
        for section in Config.sections():
            for key in Config.options(section):
                if key in self.allowable_keys:
                    val = Config.get(section, key)
                    try:
                        val = float(val)
                    except:
                        pass
                    setattr(self, key, val)       
        
        if config_file!=None:
            Config = configparser.ConfigParser()
            Config.read(config_file) # read the default config file.
        
            for section in Config.sections():
                for key in Config.options(section):
                    if key in self.allowable_keys:
                        val = Config.get(section, key)
                        try:
                            val = float(val)
                        except:
                            pass
                        setattr(self, key, val)
        
        # Read the kwargs and override params from config file.
        for key in kwargs:
            if key in self.allowable_keys:
                setattr(self, key, kwargs[key])
                
                
        n_ftr =4
        self.n_states = self.n_bins**n_ftr
        self.qtab = np.ones(())
        pass
    
    def _discretizer(e, edot, fn, fndot):
        # Discretize features and Find the corresponding element in Q table
        # bins
        pass
    
    def _role2force():
        pass
    
    def _reward2subjective_cost():
        pass
    
    def _get_role_from_q():
        # return eps-greedy(q(s))
        pass
    
    # interface
    def get_action(self, observations, update_policy=True):
        return 1
    
    def store_outcome(self, next_s, reward):
        
        pass
    
    def update_q(self, terminal=False):
        pass
    


## TD Experiment

In [18]:
class Digitizer():
    # Maybe accelerate using Cython
    # https://cython.readthedocs.io/en/latest/src/quickstart/build.html#using-the-jupyter-notebook
    
    def __init__(self, bins, max_ref, max_f):
        self.bins = max_ref*np.array(bins)
        self.max_w = 2*np.pi*max_f
        self.base = len(bins)+1
        self.n_ftr = 4
        
    def list2ind(self, numlist):
        ind = 0
        for i in range(self.n_ftr):
            ind = ind+numlist[i]* (self.base**(self.n_ftr-i-1))
        
        return ind
            
        
    def state_c2d(self, observations):
        # Form features from observations and digitize them.     
        err, edot = observations[0]-observations[2], (observations[1]-observations[3])/self.max_w
        err2, edot2 = -err, -edot
        fn, fndot = observations[4], observations[5]

#         e1_i, ed1_i, e2_i, ed2_i, fn_i, fnd_i  
        inds = np.searchsorted(self.bins, [err, edot, fn, fndot, err2, edot2])
#         inds2 = inds1; inds2[]
        s1 = self.list2ind(inds[0:4])
        s2 = self.list2ind([inds[4], inds[5], inds[2], inds[3]])

        return s1, s2
    


In [19]:
# Verification
# ind=0; numlist=[3,2,6,5]
# base = 10
# for i in range(4):
#     ind = ind+numlist[i]* (base**(4-i-1))
    
# # Benchmarking the discretization
# dd = Digitizer([-0.5, -0.1, -0.02, 0.02, 0.1, 0.5], env.max_ref, 0.5)
# q1_tab = np.random.rand(7**4, 2)
# q2_tab = np.random.rand(7**4, 2)

# t0 = time.time()
# n=int(30./0.025)
# for i in range(n):
#     obs = np.random.rand(6)
#     s1, s2 = dd.state_c2d(obs)
#     x,y = q1_tab[s1,:], q2_tab[s2,:]
# #     print('q1(s) = ', q1_tab[s1,:], '. q2(s) = ', q2_tab[s2,:])
# el_t = time.time()-t0
# print(el_t)

In [22]:
gamma = 0.7 # discount factor
alpha =0.1 #learning rate
n_td = 10 # n parameter in n-step Sarsa

env = PhysicalDyads(seed_=1234)
agent1 = TDAgent()
agent2 = TDAgent()

digitizer = Digitizer([-0.5, -0.1, -0.02, 0.02, 0.1, 0.5], env.max_ref, env.max_f)


n_episodes = 2

# ep_duration = env.get_episode_duration()
ep_true_duration = np.inf

for _ in range(n_episodes):
    st0 = env.reset(renew_traj=True)
    a1, a2 = agent1.get_action(st0), agent2.get_action(st0)
    while True:
        t = env.get_time()
        print(t)
        observations, reward, done, _ = env.step((a1, a2))
        
        # Form features and digitize them. 
        # Note: This operation is implemented outside the agents for efficiency.
        # observation={r, r', x, x', fn, f'n}
        ref = observations[0]
        s1, s2 = digitizer.state_c2d(observations)
        
        
        agent1.store_outcome(s1, reward)
        agent2.store_outcome(s2, reward)
        
        ref = observations[0]
        if env.is_terminal(ref):
            break #@@@@@@@@@@@@@
            ep_true_duration = t+1
        else:
            a1 = agent1.get_action(observations, update_policy=True)
            a2 = agent2.get_action(observations, update_policy=True)
        
        # Update Q
        tau = t-n_td+1 # tau is the time whose estimate is being updated
        
        if tau>=0: # Rules out the time steps before having enough samples.
            if tau+n_td >ep_true_duration:
                agent1.update_q(terminal=True)
                agent2.update_q(terminal=True)
            else:
                agent1.update_q()
                agent2.update_q()
        
        if tau == ep_true_duration-1:
            break
            
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27